In [ ]:
%load_ext autoreload
%autoreload 2
%aimport utils_1_1

import pandas as pd
import numpy as np
import altair as alt
from altair_saver import save

from constants_1_1 import SITE_FILE_TYPES
from utils_1_1 import (
    read_loinc_df,
    get_site_file_paths,
    get_site_file_info,
    get_site_ids,
    read_full_med_df,
    get_visualization_subtitle,
    apply_theme,
    get_anonymousid_color_maps
)
from web import for_website

alt.data_transformers.disable_max_rows() # Allow using rows more than 5000

In [ ]:
original_df = pd.read_csv("../data/Figure_678_med_plot_meta_metafor.csv", sep=",", header=0)
original_df

In [ ]:
df = original_df.copy()

# rename columns
df = df.rename(columns={
    'siteId': 'siteid',
    'time': 'before_or_since',
    'medname': 'med'
})

# rename categories
df.before_or_since = df.before_or_since.apply(lambda x: 'Before admission' if x == 'before_admission' else 'After admission')

# filter rows
df = df[df.siteid == 'pool']

# remove columns we don't use
df = df.drop(columns=['siteid'])

df

In [ ]:
MEDICATIONS = df['med'].unique().tolist()
MEDICATIONS

In [ ]:
dropdown = alt.binding_select(options=MEDICATIONS)
selection = alt.selection_single(fields=['med'], bind=dropdown, name='Medication', init={'med': MEDICATIONS[0]})

tick_size = 40

base = (
    alt.Chart(df)
        .mark_bar(size=tick_size)
        .encode(
            x=alt.Y('before_or_since:N', scale=alt.Scale(domain=['Before admission', 'After admission'])),
            y=alt.Y('ci_95L:Q', title='Pooled mean (CI)'),
            y2=alt.Y2('ci_95U:Q'),
            color=alt.Color(
                'before_or_since:N', 
                title=None, 
                scale=alt.Scale(
                    domain=['Before admission', 'After admission'], 
                    range=['#DC3A11', '#3366CC']
                ),
                legend=None
            ),
            stroke=alt.Color('before_or_since:N', legend=None),
            strokeWidth=alt.value(1),
            tooltip=[
                alt.Tooltip("before_or_since", title="Before or Since"),
                alt.Tooltip("mean", title="Mean", format=".2f"),
                alt.Tooltip("ci_95L", title="CI (.95 Lower)", format=".2f"),
                alt.Tooltip("ci_95U", title="CI (.95 Upper)", format=".2f")
            ]
        )
)

mean = alt.Chart(df).mark_tick(size=tick_size, thickness=2).encode(
    x=alt.X('before_or_since:N', title=None),
    opacity=alt.value(1),
    color=alt.value('white'),
    y=alt.Y('mean:Q')
)

subtitle = get_visualization_subtitle(data_release='06-24-2020', with_num_sites=False) + '  |  25 Sites'

plot = alt.layer(
    base, mean
).add_selection(
    selection
).transform_filter(
    selection
).properties(
    title={
        "text": ["Meta-Analysis for Risk Difference"],
        "subtitle": [
            subtitle,
            '',
            '*Risk Difference = ',
            'Risk of Being Severe with Medication –', 
            'Risk of Being Severe without Medication',
        ],
        "subtitleColor": "gray",
        "anchor": "middle",
    },
    width=350, height=400
)

plot = apply_theme(plot).configure_axis(
    labelAngle=0,
    #
    labelFontSize=14,
    labelFontWeight=300,
    titleFontSize=16,
    titleFontWeight=300,
    titlePadding=16
)

for_website(plot, "1.1.Med", "Medication risk difference")

plot